In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

## Downloading the data

We can download the data set from Amazon S3:

In [ ]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

--2019-11-14 16:30:47--  http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.216.161.139
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.216.161.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3398120 (3.2M) [text/csv]
Saving to: ‘./ml-data/providers-train.csv’

     0K .......... .......... .......... .......... ..........  1%  276K 12s
    50K .......... .......... .......... .......... ..........  3% 2.00M 7s
   100K .......... .......... .......... .......... ..........  4%  744K 6s
   150K .......... .......... .......... .......... ..........  6% 77.0M 4s
   200K .......... .......... .......... .......... ..........  7%  555K 4s
   250K .......... .......... .......... .......... ..........  9% 41.3M 4s
   300K .......... .......... .......... .......... .......... 10% 57.6M 3s
   350K .......... ..........

We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [ ]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}
        self.global_mean = 0

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.global_mean = y.mean()
        return self

    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
        #predictions = [self.group_averages[group] for group in X[self.grouper]]
        if isinstance(X, list):
            X = pd.DataFrame(X)
        predictions = [self.group_averages.get(group, self.global_mean)
                      for group in X[self.grouper]]
        return predictions

In [ ]:
fine_dict = fine_totals.groupby(data['STATE']).mean().to_dict()
fine_dict

{'AK': 15932.75,
 'AL': 13672.320388349515,
 'AR': 17596.6815920398,
 'AZ': 1512.7222222222222,
 'CA': 8054.977611940299,
 'CO': 22112.54591836735,
 'CT': 8438.121359223302,
 'DC': 27333.933333333334,
 'DE': 24899.0,
 'FL': 16612.338211382114,
 'GA': 29459.975,
 'GU': 0.0,
 'HI': 16133.309523809523,
 'IA': 16565.303571428572,
 'ID': 42741.942028985504,
 'IL': 6634.197226502311,
 'IN': 5626.954545454545,
 'KS': 24420.79109589041,
 'KY': 32656.315384615384,
 'LA': 5611.819277108434,
 'MA': 16722.25925925926,
 'MD': 42806.67661691542,
 'ME': 1275.5869565217392,
 'MI': 21437.39746835443,
 'MN': 3219.326530612245,
 'MO': 7635.160997732426,
 'MS': 7595.4812834224595,
 'MT': 32754.970588235294,
 'NC': 30445.040506329115,
 'ND': 560.171052631579,
 'NE': 8377.755319148937,
 'NH': 260.45588235294116,
 'NJ': 3490.756838905775,
 'NM': 39652.64705882353,
 'NV': 3050.1964285714284,
 'NY': 2213.51526032316,
 'OH': 8214.822977725675,
 'OK': 11812.111111111111,
 'OR': 12365.983606557376,
 'PA': 9216.96

In [ ]:
fine_dict.get('AK',0)

15932.75

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [ ]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [ ]:
sample = data.sample(5)
state_model.predict(sample)

[16612.338211382114,
 16612.338211382114,
 8214.822977725675,
 16565.303571428572,
 9216.964686998395]

In [ ]:
sample['STATE']

2134     FL
1966     FL
9092     OH
4096     IA
10405    PA
Name: STATE, dtype: object

In [ ]:
#state_model.predict(pd.DateFrame)

state_model.predict(data)

[13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [ ]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [ ]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

In [ ]:
from sklearn.preprocessing import Imputer

In [ ]:

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]

simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('impute', Imputer())
])

(Note that the assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [ ]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [ ]:


Machine learning models as classes
Scikit-learn relies heavily on object-oriented programming principles. It implements machine learning algorithms as classes and users create objects from these "recipes". For example, Ridge is a class representing the ridge regression model. To create a Ridfrom sklearn.ensemble import RandomForestClassifier

SyntaxError: invalid syntax (<ipython-input-21-516fb7529c35>, line 3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_dict = {
    'n_estimators': range(150,251,50),
    'min_samples_leaf': range(100,151,25),
    'class_weight': ["balanced", "balanced_subsample"]
}

grid_search = GridSearchCV(RandomForestClassifier(),
                          param_grid = param_dict,
                          cv = 3,
                          n_jobs = 3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

simple_features_model = Pipeline([
    ('simple', simple_features),
    ('gs', grid_search)
    #('rf', RandomForestClassifier())
    # add your estimator here
])

In [ ]:
simple_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
     steps=[('simple', Pipeline(memory=None,
     steps=[('cst', ColumnSelectTransformer(columns=['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL'])), ('impute', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [ ]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score:  0.9759083237096917


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [ ]:
data.shape

(13892, 29)

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

class CustomLabelEncoder(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.labelers = {col: LabelEncoder().fit(X[col]) for col in X}
        return self

        #return super().fit_transform(y.values.ravel()).reshape(-1,1)

    def transform(self, X):
        return pd.DataFrame({col: self.labelers[col].transform(X[col]) for col in X})

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('cle', CustomLabelEncoder()),
    ('onehot', OneHotEncoder(sparse = False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('cle', CustomLabelEncoder()),
    #('labelenc', LabelEncoder),
    ('onehot', OneHotEncoder(sparse = False))
])

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [ ]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
param_dict = {
    'n_estimators': range(150,251,50),
    'min_samples_leaf': range(100,151,25),
    'class_weight': ["balanced", "balanced_subsample"]
}

grid_search = GridSearchCV(RandomForestClassifier(),
                          param_grid = param_dict,
                          cv = 3,
                          n_jobs = 3)

In [ ]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    ('gs', grid_search)
    #('rf', RandomForestClassifier())
    # add your estimator here
])

In [ ]:
categorical_features_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
     steps=[('categorical', FeatureUnion(n_jobs=1,
       transformer_list=[('owner', Pipeline(memory=None,
     steps=[('cst', ColumnSelectTransformer(columns=['OWNERSHIP'])), ('cle', CustomLabelEncoder()), ('onehot', OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [ ]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  1.0031272337383836


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [ ]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [ ]:
param_dict = {
    'n_estimators': range(150,251,50),
    'min_samples_leaf': range(100,151,25),
    'class_weight': ["balanced", "balanced_subsample"]
}

grid_search = GridSearchCV(RandomForestClassifier(),
                          param_grid = param_dict,
                          cv = 3,
                          n_jobs = 3)

In [ ]:

business_model = Pipeline([
    ('features', business_features),
    ('gs', grid_search)
    #('rf', RandomForestClassifier())
    # add your estimator here
])

In [ ]:
business_model.fit(data, fine_counts > 0)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('simple', Pipeline(memory=None,
     steps=[('cst', ColumnSelectTransformer(columns=['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL'])), ('impute', Imputer(axis=0,...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [ ]:
grader.score.ml__business_model(positive_probability(business_model))

Your score:  0.9247717281028063


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [ ]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X,pd.DataFrame):
            X = pd.DataFrame(X)
        return (pd.to_datetime(X[self.t2_col]) -
                pd.to_datetime(X[self.t1_col])).values.reshape(-1,1).astype(float)

In [ ]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In [ ]:
(pd.to_datetime(data['CYCLE_2_SURVEY_DATE']) - pd.to_datetime(data['CYCLE_1_SURVEY_DATE'])).values.reshape(-1,1).astype(float)

array([[-2.72160e+16],
       [-3.50784e+16],
       [-2.54880e+16],
       ...,
       [-2.48832e+16],
       [-2.85984e+16],
       [-8.46720e+15]])

In the cell below we'll collect the cycle 1 survey features.

In [ ]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD

param_dict = {'n_estimators' : range(174,176,1),
             'min_samples_leaf' : range(40,55,3),
             'min_samples_split': range(2,6)}

gs = GridSearchCV(
    RandomForestRegressor(),
    param_grid = param_dict,
    cv = 3,
    n_jobs = 4
)

survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    # add your estimator here
    ('poly', PolynomialFeatures(2)),
    ('svd', TruncatedSVD()),
    ('gs', gs),
    #('rfr', RandomForestRegressor(150)),
])

In [ ]:
survey_model.named_steps['gs'].best_params_

{'min_samples_leaf': 45, 'min_samples_split': 4, 'n_estimators': 174}

In [ ]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('business', FeatureUnion(n_jobs=1,
       transformer_list=[('simple', Pipeline(memory=None,
     steps=[('cst', ColumnSelectTransformer(columns=['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STA...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))])

In [ ]:
grader.score.ml__survey_model(survey_model.predict)

Your score:  0.904895594088456


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*